In [1]:
import numpy as np

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator

## 1. Controlled rotational gates

Find the circuit decompisition for $CR_x(\theta)$. You can use the code below to check whether your decomposition is correct (in which case there should be no "not equal" prints).

In [29]:
circ1 = QuantumCircuit(2) # your quantum circuit

circ1.rx(np.pi, 1)
circ1.cx(0,1)
circ1.rz(0.62, 1)
circ1.cx(0,1)
circ1.rx(-np.pi, 1)

print(circ1.draw())
print(Operator(circ1).data)

state = Statevector(circ1)
state.draw('latex')

                                              
q_0: ───────────■────────────────■────────────
     ┌───────┐┌─┴─┐┌──────────┐┌─┴─┐┌────────┐
q_1: ┤ Rx(π) ├┤ X ├┤ Rz(0.62) ├┤ X ├┤ Rx(-π) ├
     └───────┘└───┘└──────────┘└───┘└────────┘
[[ 9.52333570e-01+0.30505864j  0.00000000e+00+0.j
  -3.73589083e-17+0.j          0.00000000e+00+0.j        ]
 [ 0.00000000e+00+0.j          9.52333570e-01-0.30505864j
   0.00000000e+00+0.j          3.73589083e-17+0.j        ]
 [ 3.73589083e-17+0.j          0.00000000e+00+0.j
   9.52333570e-01-0.30505864j  0.00000000e+00+0.j        ]
 [ 0.00000000e+00+0.j         -3.73589083e-17+0.j
   0.00000000e+00+0.j          9.52333570e-01+0.30505864j]]


<IPython.core.display.Latex object>

In [30]:
circ2 = QuantumCircuit(2)
circ2.crx(0.62, 0, 1)
print(circ2.draw())

print(Operator(circ2).data)

                 
q_0: ─────■──────
     ┌────┴─────┐
q_1: ┤ Rx(0.62) ├
     └──────────┘
[[1.        +0.j         0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.95233357+0.j         0.        +0.j
  0.        -0.30505864j]
 [0.        +0.j         0.        +0.j         1.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        -0.30505864j 0.        +0.j
  0.95233357+0.j        ]]


In [31]:
Operator(circ2).data == Operator(circ1).data

array([[False,  True, False,  True],
       [ True, False,  True, False],
       [False,  True, False,  True],
       [ True, False,  True, False]])

In [33]:
for i in range(4):
    for j in range(4):
        if Operator(circ1).data[i][j] - Operator(circ2).data[i][j] > 1e-15:
            print("not equal")


## 2. Measuring composite operators

Consider the following quantum circuits.

In [8]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from numpy import pi

qreg_q = QuantumRegister(2, 'q')
circuit = QuantumCircuit(qreg_q)

circuit.h(qreg_q[0])
circuit.ry(pi / 6, qreg_q[1])
circuit.cx(qreg_q[0], qreg_q[1])

circuit.draw()

┌───┐        
q_0: ───┤ H ├─────■──
     ┌──┴───┴──┐┌─┴─┐
q_1: ┤ Ry(π/6) ├┤ X ├
     └─────────┘└───┘
c: 4/════════════════

This circuit acts on two qubits and produce a quantum state which is a reasonable ground state approxiamtion of the two-site (ferromagnetic) quantum transverse field Ising Hamiltonian at the critical point, defined as
$$H = - Z Z - X I - I X$$

1. Find the matrix representation of the Hamiltonian H.

$$ Z = \begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix} \quad
X = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix} \quad
I = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix} \quad
$$

$$ ZZ = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & -1 & 0 & 0 \\ 0 & 0 & -1 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix} \quad
XI = \begin{bmatrix} 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \\ 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \end{bmatrix} \quad
IX = \begin{bmatrix} 0 & 1 & 0 & 0 \\ 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{bmatrix} \quad
$$


thus:

$$ H = -ZZ - XI - IX = \begin{bmatrix} -1 & -1 & -1 & 0 \\ -1 & 1 & 0 & -1 \\ -1 & 0 & 1 & -1 \\ 0 & -1 & -1 & 1 \end{bmatrix} \quad $$

2. Find the statevector represantion $|\psi \rangle$ of the wavefunction generated by the above circuit (acting on the $|00\rangle$ state)

$$ \begin{bmatrix} 1 \\ 0 \\ 0 \\ 0 \end{bmatrix} \times H = \begin{bmatrix} -1 \\ -1 \\ -1 \\ 0 \end{bmatrix} \quad = -|00\rangle - |01\rangle - |10\rangle$$



3. Compute the expectation value of the energy, exactly: $\langle \psi | H | \psi \rangle$

$$\hat{H} = \langle \psi | H | \psi \rangle = \begin{bmatrix} \psi^*_0 & \psi^*_1 & \psi^*_2 & \psi^*_3 \end{bmatrix} \times \begin{bmatrix} -1 & -1 & -1 & 0 \\ -1 & 1 & 0 & -1 \\ -1 & 0 & 1 & -1 \\ 0 & -1 & -1 & 1 \end{bmatrix} \times \begin{bmatrix} \psi_0 \\ \psi_1 \\ \psi_2 \\ \psi_3 \end{bmatrix} $$

$$ \begin{bmatrix} \psi^*_0 & \psi^*_1 & \psi^*_2 & \psi^*_3 \end{bmatrix} \times \begin{bmatrix} -1 \\ -1 \\ -1 \\ 0 \end{bmatrix} = -\psi^*_0 - \psi^*_1 - \psi^*_2 $$

4. Find a procedure to compute the expectation value of the energy using the quantum measurements.
5. bonus: what's the minimal number of circuits needed in this case